In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os

In [2]:
#setting path
path = r"C:\Users\luiss\OneDrive\Career Foundry\Instacart Basket Analysis"

In [3]:
#importing data
df_opm = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','ords_prods_merge.pkl'))

In [4]:
df_opm.shape

(32406041, 18)

In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [5]:
#Finding mean of order number grouped by department id
df_opm.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.213779
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.171095
12    15.887671
13    16.583536
14    16.771241
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

My subset results were different than given in the reading, and I did double check using shape to make sure I had subsetted to the first million rows, I'm guessing the difference is due to organization.  That being said, it seems like I got a good sample size from the initial million since the largest percent difference is 7%, with a majority being under 3% difference.  Both the initial million and complete data set featured all departments.

Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [6]:
#Following cells of code will be to differentiate customers into Loyal, Regular, and New depending on the amount of orders they've made
#Creating max order column

df_opm['max_order'] = df_opm.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\luiss\AppData\Local\Temp\ipykernel_1980\3142672560.py:4: FutureWarning: The provided callable <function max at 0x00000195651784A0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_opm['max_order'] = df_opm.groupby(['user_id'])['order_number'].transform(np.max)


In [7]:
df_opm.head()

order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   
3   2539329        1             1                   2                  8   
4   2539329        1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered _merge  \
0                     NaN         196                  1          0   both   
1                     NaN       14084                  2          0   both   
2                     NaN       12427                  3          0   both   
3                     NaN       26088                  4          0   both   
4                     NaN       26405                  5          0   both   

   Unnamed: 0                             product_name  aisle_id  \
0         195                                     Soda        77   
1       14084  Organic Unsweetened Vanilla Almond Milk        91   
2       12427                      Original Beef Jerky        23   
3       26089               Aged White Cheddar Popcorn        23   
4       26406         XL Pick-A-Size Paper Towel Rolls        54   

   department_id  prices    price_range_loc     busiest_day  \
0              7     9.0  Mid-range product  Regularly busy   
1             16    12.5  Mid-range product  Regularly busy   
2             19     4.4  Low-range product  Regularly busy   
3             19     4.7  Low-range product  Regularly busy   
4             17     1.0  Low-range product  Regularly busy   

  busiest_period_of_day  max_order  
0        Average orders         10  
1        Average orders         10  
2        Average orders         10  
3        Average orders         10  
4        Average orders         10

In [8]:
df_opm.groupby('user_id')['max_order'].mean()

user_id
1         10.0
2         14.0
3         12.0
4          5.0
5          4.0
          ... 
206205     3.0
206206    67.0
206207    16.0
206208    49.0
206209    13.0
Name: max_order, Length: 206209, dtype: float64

In [9]:
#Assigning labels dependent on order numbers

df_opm.loc[df_opm['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [10]:
df_opm.loc[(df_opm['max_order'] <=40) & (df_opm['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [11]:
df_opm.loc[df_opm['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
df_opm['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15877395
Loyal customer      10284396
New customer         6244250
Name: count, dtype: int64

The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [13]:
#Finding mean price spent by customer organized by loyalty and department

df_opm.groupby(['loyalty_flag','department_id'])['prices'].mean()

loyalty_flag      department_id
Loyal customer    1                7.663262
                  2                6.743136
                  3                7.883289
                  4                7.950691
                  5                8.159503
                                     ...   
Regular customer  17               7.375086
                  18               7.671737
                  19               4.279951
                  20               7.771021
                  21               8.722086
Name: prices, Length: 63, dtype: float64

In [14]:
#setting max rows to 63 to see entire previous table

pd.options.display.max_rows = 63

In [15]:
#Simplifying above
df_opm.groupby('loyalty_flag')['prices'].mean()

loyalty_flag
Loyal customer      10.386229
New customer        13.294406
Regular customer    12.495501
Name: prices, dtype: float64

Initially I thought to organize by department as well since different departments will have different costs but then I realized I'd have to look at total amounts spent as well and that I might've been over-answering a question I'll be looking more into later anyways.  So, per order, regular customers spend the most with loyal customers spending the least.  If loyal customers are like my wife and I, they get use emails/apps/mail to learn about when stuff is on sale.

The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [16]:
#Creating new column for average price

df_opm['average_price'] = df_opm.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\luiss\AppData\Local\Temp\ipykernel_1980\3707097205.py:3: FutureWarning: The provided callable <function mean at 0x0000019565178EA0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_opm['average_price'] = df_opm.groupby(['user_id'])['prices'].transform(np.mean)


In [17]:
df_opm.head()

order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   
3   2539329        1             1                   2                  8   
4   2539329        1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered _merge  \
0                     NaN         196                  1          0   both   
1                     NaN       14084                  2          0   both   
2                     NaN       12427                  3          0   both   
3                     NaN       26088                  4          0   both   
4                     NaN       26405                  5          0   both   

   ...                             product_name aisle_id  department_id  \
0  ...                                     Soda       77              7   
1  ...  Organic Unsweetened Vanilla Almond Milk       91             16   
2  ...                      Original Beef Jerky       23             19   
3  ...               Aged White Cheddar Popcorn       23             19   
4  ...         XL Pick-A-Size Paper Towel Rolls       54             17   

   prices    price_range_loc     busiest_day busiest_period_of_day max_order  \
0     9.0  Mid-range product  Regularly busy        Average orders        10   
1    12.5  Mid-range product  Regularly busy        Average orders        10   
2     4.4  Low-range product  Regularly busy        Average orders        10   
3     4.7  Low-range product  Regularly busy        Average orders        10   
4     1.0  Low-range product  Regularly busy        Average orders        10   

   loyalty_flag average_price  
0  New customer      6.367797  
1  New customer      6.367797  
2  New customer      6.367797  
3  New customer      6.367797  
4  New customer      6.367797  

[5 rows x 21 columns]

In [18]:
#Will categorize user_ids based on average product price in low spender and high spender

df_opm.loc[df_opm['average_price'] < 10, 'spending_flag'] = 'Low spender'

In [19]:
df_opm.loc[df_opm['average_price'] >= 10, 'spending_flag'] = 'High spender'

In [20]:
df_opm['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31771789
High spender      634252
Name: count, dtype: int64

In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [21]:
#Creating new column based on median of days since prior order

df_opm['median_days_since_prior_order'] = df_opm.groupby('user_id')['days_since_prior_order'].transform(np.median)

C:\Users\luiss\AppData\Local\Temp\ipykernel_1980\1673869508.py:3: FutureWarning: The provided callable <function median at 0x000001956529BB00> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_opm['median_days_since_prior_order'] = df_opm.groupby('user_id')['days_since_prior_order'].transform(np.median)


In [29]:
df_opm.head()

order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   
3   2539329        1             1                   2                  8   
4   2539329        1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered _merge  \
0                     NaN         196                  1          0   both   
1                     NaN       14084                  2          0   both   
2                     NaN       12427                  3          0   both   
3                     NaN       26088                  4          0   both   
4                     NaN       26405                  5          0   both   

   ...  prices    price_range_loc     busiest_day  busiest_period_of_day  \
0  ...     9.0  Mid-range product  Regularly busy         Average orders   
1  ...    12.5  Mid-range product  Regularly busy         Average orders   
2  ...     4.4  Low-range product  Regularly busy         Average orders   
3  ...     4.7  Low-range product  Regularly busy         Average orders   
4  ...     1.0  Low-range product  Regularly busy         Average orders   

   max_order  loyalty_flag average_price spending_flag  \
0         10  New customer      6.367797   Low spender   
1         10  New customer      6.367797   Low spender   
2         10  New customer      6.367797   Low spender   
3         10  New customer      6.367797   Low spender   
4         10  New customer      6.367797   Low spender   

   median_days_since_prior_order         frequency_flag  
0                           20.5  Non-frequent customer  
1                           20.5  Non-frequent customer  
2                           20.5  Non-frequent customer  
3                           20.5  Non-frequent customer  
4                           20.5  Non-frequent customer  

[5 rows x 24 columns]

In [30]:
#Creating new flag based on non-frequent, regular, and frequent customers based on median days since prior order

df_opm.loc[df_opm['median_days_since_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [31]:
df_opm.loc[(df_opm['median_days_since_prior_order'] > 10) & (df_opm['median_days_since_prior_order'] <= 20), 'frequency_flag'] = 'Regular customer'

In [32]:
df_opm.loc[df_opm['median_days_since_prior_order'] <= 10, 'frequency_flag'] = 'Frequenct customer'

In [33]:
df_opm['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequenct customer       21560324
Regular customer          7209005
Non-frequent customer     3636707
NaN                             5
Name: count, dtype: int64

In [36]:
df_opm['days_since_prior_order'].value_counts(dropna = False)

days_since_prior_order
7.0     3476307
30.0    3067482
6.0     2517580
5.0     2124508
4.0     2078699
NaN     2076201
8.0     1932026
3.0     1876248
2.0     1463774
9.0     1217894
14.0    1029690
10.0     967224
1.0      940508
13.0     851152
11.0     820791
12.0     774192
15.0     671488
16.0     461352
0.0      448335
21.0     444160
17.0     382129
20.0     379743
18.0     350615
19.0     334899
22.0     308862
28.0     253163
23.0     225841
27.0     204071
24.0     193429
25.0     180423
26.0     177778
29.0     175477
Name: count, dtype: int64

In [37]:
#Exporting data

df_opm.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))